In [30]:
from selenium import webdriver
import re
from bs4 import BeautifulSoup 
import time
import pandas as pd

In [31]:
driver = webdriver.Chrome(r'C:\Users\82105\Desktop\chromedriver.exe') 
driver.get('https://www.instagram.com')
time.sleep(2)

# 아이디 입력
email = 'funfun.data0813@gmail.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

# 비밀번호 입력
password = 'dftn7jj7'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.send_keys(password)
input_pw.submit()

# 알림창 버튼 (나중에저장 클릭)
time.sleep(2)
close = driver.find_element_by_css_selector('div.cmbtv')
close.click()

# 알림창 버튼 2 (알림 설정)
time.sleep(2)
close = driver.find_element_by_css_selector('button.aOOlW.HoLwm')
close.click()

In [32]:

def insta_searching(word):
    url = 'https://www.instagram.com/explore/tags/' + word
    return url

In [33]:
word = '군산맛집'
url = insta_searching(word)
driver.get(url)

In [34]:

def select_first(driver):
    first = driver.find_element_by_css_selector("div.v1Nh3.kIKUG._bz0w")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [35]:
def get_content(driver):

    # 현재 페이지 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    
    # 본문 내용 가져오기
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content=' '
        
    # 본문 내용에서 해시태그 가져오기 (정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)
    
    # 작성일자 정보 가져오기
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]

    # 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]
    except:
        like=0
        
    # 위치정보가져오기
    try:
        place = soup.select('div.M30cS')[0].text
    except:
        place=''
        
    # 수집한 정보 저장하기
    data = [content, date, like, place, tags]
    return data

get_content(driver)

# like, place 두개 수집이 안됨

['.🍊 군산 맛집..🍊 정보.- 이름 : 장춘횟집- 시간 : 09 : 00 ~ 21 : 30- 주소 : 전라북도 군산시 내항2길 153.🍊 후기.- 군산에 가면 항상 생각이 나는 곳이에요.- 한상 차림이 이렇게 푸짐해도 되나 싶을 정도로 푸짐했어요.- 양도 많은 편이긴 하지만 퀄리티가 너무 좋아서 항상 만족해요.- 입에서 살살 녹는 맛을 느낄 수가 있어요.🍊 N 행시.- 이 : 이새끼야- 성 : 성실하게 일 안 할래?- 당 : 당장 일 해라 제발.',
 '2021-07-22',
 0,
 'Gunsan',
 []]

In [36]:
def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(2)
    
move_next(driver)

인스타그램 크롤링

In [54]:
driver = webdriver.Chrome(r'C:\Users\82105\Desktop\chromedriver.exe') 
driver.get('https://www.instagram.com')
time.sleep(2)

email = 'funfun.data0813@gmail.com'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'dftn7jj7'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

# 알림창 버튼 (나중에저장 클릭)
time.sleep(5)
close = driver.find_element_by_css_selector('div.cmbtv')
close.click()

# 알림창 버튼 2 (알림 설정)
time.sleep(5)
close2 = driver.find_element_by_css_selector('button.aOOlW.HoLwm')
close2.click()

word = '군산맛집추천'
url = insta_searching(word)

driver.get(url)
time.sleep(10)

select_first(driver)

results = []

target = 100 # 크롤링할 게시글 수
for i in range(target):
    data = get_content(driver)
    results.append(data)
    move_next(driver)
    
print(results[:2])

IndexError: list index out of range

In [55]:
results_df =pd.DataFrame(results)
results_df.columns=['content','data','like','place','tags']
results_df.to_excel('./군산맛집추천크롤링.xlsx')

In [51]:
jeju_insta_df = pd.DataFrame( [ ] )

folder = 'C:/Users/82105/Desktop/insta/'
f_list = ['YeoHang.xlsx', 'MatJip.xlsx', 'GwanGwang.xlsx', '2MatJip.xlsx']
for fname in f_list:
    fpath = folder + fname
    temp = pd.read_excel(fpath)
    jeju_insta_df = jeju_insta_df.append(temp)

jeju_insta_df.columns =['no','content','data','like','place','tags']

In [52]:
jeju_insta_df.drop_duplicates(subset=['content'], inplace=True)
jeju_insta_df.to_excel('./4.1 crawling_raw.xlsx', index=False)